In [3]:
# check_shp.py
import geopandas as gpd

SHP_PATH = r"D:\Project\02_지도시각화\지도\국가기본도_시군구구역경계\TN_SIGNGU_BNDRY.shp"

gdf = gpd.read_file(SHP_PATH, encoding="cp949")  # 안 되면 encoding 빼도 됨

print("컬럼 목록:", gdf.columns)
print("CRS(좌표계):", gdf.crs)
print(gdf.head(3))

컬럼 목록: Index(['NF_ID', 'ADZONE_NM', 'LEGLCD_SE', 'ADZONE_SE', 'OBCHG_DT',
       'MESRMTH_SE', 'RSREG_DT', 'MNENT_NM', 'DBREG_DT', 'geometry'],
      dtype='object')
CRS(좌표계): EPSG:5179
               NF_ID ADZONE_NM   LEGLCD_SE ADZONE_SE   OBCHG_DT MESRMTH_SE  \
0  ARB0200000000001F       속초시  5121000000    ADD006 2024-04-17          P   
1  ARB0200000000002G       인제군  5181000000    ADD007 2024-04-17          P   
2  ARB0200000000003H       고성군  5182000000    ADD007 2024-04-17          P   

    RSREG_DT    MNENT_NM   DBREG_DT  \
0 2024-04-17  (주)우리강산시스템 2024-04-17   
1 2024-04-17  (주)우리강산시스템 2024-04-17   
2 2024-04-17  (주)우리강산시스템 2024-04-17   

                                            geometry  
0  MULTIPOLYGON (((1097616.57 2020526.167, 109761...  
1  MULTIPOLYGON (((1088467.239 2001854.857, 10884...  
2  MULTIPOLYGON (((1094183.073 2028131.872, 10941...  


In [1]:
import geopandas as gpd
import folium
import os

# 1. 파일 경로 설정 (notebooks 폴더에서 한 단계 위로 올라가서(../) 지도 폴더로 진입)
shp_path = '../지도/국가기본도_시군구구역경계/TN_SIGNGU_BNDRY.shp'

# 파일이 잘 잡히는지 확인
if os.path.exists(shp_path):
    print(f"파일을 찾았습니다: {shp_path}")
else:
    print(f"❌ 경로를 확인해주세요. 현재 설정된 경로: {shp_path}")

# 2. Shapefile 읽기 (인코딩은 보통 cp949)
gdf = gpd.read_file(shp_path, encoding='cp949')

# 3. 데이터 확인 (컬럼명 확인이 제일 중요해!)
print("\n--- 컬럼 목록 ---")
print(gdf.columns)
print("\n--- 데이터 샘플 (상위 3개) ---")
display(gdf.head(3))

# 4. 대구 지역 필터링 및 좌표계 변환
# 보통 시군구코드는 'SIGNGU_CD' 또는 'SIG_CD' 입니다. (데이터 출력 결과 보고 수정 필요할 수도 있음)
# 대구 코드는 '27'로 시작
target_col = 'SIGNGU_CD' # 만약 에러나면 위에서 출력된 컬럼명 보고 'SIG_CD' 등으로 바꿔줘!

if target_col in gdf.columns:
    daegu_gdf = gdf[gdf[target_col].str.startswith('27')].copy()
    print(f"\n📍 대구 지역 {len(daegu_gdf)}개 구역 추출 완료")
    
    # 좌표계 정의 및 변환 (국가기본도는 보통 EPSG:5179 사용)
    if daegu_gdf.crs is None:
        daegu_gdf.set_crs("EPSG:5179", inplace=True) # 좌표계 정보가 파일에 없으면 강제 지정
    
    # 지도에 표시하려면 위경도(EPSG:4326)로 무조건 변환해야 함
    daegu_gdf = daegu_gdf.to_crs(epsg=4326)
else:
    print(f"❌ '{target_col}' 컬럼을 못 찾았어요. 위 컬럼 목록을 확인해주세요.")

파일을 찾았습니다: ../지도/국가기본도_시군구구역경계/TN_SIGNGU_BNDRY.shp

--- 컬럼 목록 ---
Index(['NF_ID', 'ADZONE_NM', 'LEGLCD_SE', 'ADZONE_SE', 'OBCHG_DT',
       'MESRMTH_SE', 'RSREG_DT', 'MNENT_NM', 'DBREG_DT', 'geometry'],
      dtype='object')

--- 데이터 샘플 (상위 3개) ---


,NF_ID,ADZONE_NM,LEGLCD_SE,ADZONE_SE,OBCHG_DT,MESRMTH_SE,RSREG_DT,MNENT_NM,DBREG_DT,geometry
0,ARB0200000000001F,속초시,5121000000,ADD006,2024-04-17,P,2024-04-17,(주)우리강산시스템,2024-04-17,"MULTIPOLYGON (((1097616.57 2020526.167, 109761..."
1,ARB0200000000002G,인제군,5181000000,ADD007,2024-04-17,P,2024-04-17,(주)우리강산시스템,2024-04-17,"MULTIPOLYGON (((1088467.239 2001854.857, 10884..."
2,ARB0200000000003H,고성군,5182000000,ADD007,2024-04-17,P,2024-04-17,(주)우리강산시스템,2024-04-17,"MULTIPOLYGON (((1094183.073 2028131.872, 10941..."


❌ 'SIGNGU_CD' 컬럼을 못 찾았어요. 위 컬럼 목록을 확인해주세요.


In [6]:
import geopandas as gpd
import folium
import os
import webbrowser

# ==========================================
# 1. 파일 경로 설정
# ==========================================
# 분석.ipynb 파일 기준 상대 경로
shp_path = '../지도/국가기본도_시군구구역경계/TN_SIGNGU_BNDRY.shp'

# 파일 있는지 체크
if not os.path.exists(shp_path):
    print(f"❌ 파일을 찾을 수 없습니다. 경로를 확인해주세요: {shp_path}")
else:
    print(f"✅ 파일을 찾았습니다! 데이터를 불러옵니다...")

    # ==========================================
    # 2. 데이터 읽기 및 전처리
    # ==========================================
    # 한글 깨짐 방지를 위해 encoding='cp949' 사용
    gdf = gpd.read_file(shp_path, encoding='cp949')

    # 대구 지역 필터링 (법정동코드 'LEGLCD_SE'가 '27'로 시작하는 것)
    # 아까 보여주신 컬럼명(LEGLCD_SE)을 기준으로 작성했습니다.
    daegu_gdf = gdf[gdf['LEGLCD_SE'].str.startswith('27')].copy()
    
    print(f"📍 추출된 구역 수: {len(daegu_gdf)}개")

    # 🚨 [핵심] 날짜 타입 에러 방지 (모든 컬럼을 문자로 변환)
    for col in daegu_gdf.columns:
        if col != 'geometry':
            daegu_gdf[col] = daegu_gdf[col].astype(str)

    # 좌표계 변환 (UTM-K -> 위경도)
    # 국가기본도 원본 좌표계(EPSG:5179)를 지정하고, 지도용(EPSG:4326)으로 변환
    if daegu_gdf.crs is None:
        daegu_gdf.set_crs("EPSG:5179", inplace=True)
    
    daegu_gdf = daegu_gdf.to_crs(epsg=4326)

    # ==========================================
    # 3. 지도 생성 및 꾸미기
    # ==========================================
    # 대구 중심 좌표
    m = folium.Map(location=[35.8714, 128.6014], zoom_start=11)

    # 색상 팔레트 (구역별로 다른 색)
    colors = ['#FF0000', '#FF8C00', '#FFD700', '#008000', '#0000FF', '#4B0082', '#9400D3', '#FF1493', '#00CED1']

    # 지도에 얹기
    folium.GeoJson(
        daegu_gdf,
        style_function=lambda feature: {
            # feature['id'] 순서대로 색상을 가져옵니다
            'fillColor': colors[int(feature['id']) % len(colors)] if str(feature['id']).isdigit() else '#3388ff',
            'color': 'black',       # 테두리 색상
            'weight': 1.5,          # 테두리 두께
            'fillOpacity': 0.6      # 면 투명도
        },
        # 마우스 올렸을 때 뜰 이름 (ADZONE_NM: 행정구역명)
        tooltip=folium.GeoJsonTooltip(fields=['ADZONE_NM'], aliases=['지역명:'])
    ).add_to(m)

    # ==========================================
    # 4. 저장 및 브라우저 열기
    # ==========================================
    save_path = 'daegu_map_final.html'
    m.save(save_path)
    
    print(f"✅ 지도가 '{save_path}'로 저장되었습니다.")
    print("🚀 브라우저를 실행합니다...")
    
    # 웹 브라우저 새 탭에서 열기
    webbrowser.open(save_path)

✅ 파일을 찾았습니다! 데이터를 불러옵니다...
📍 추출된 구역 수: 19개
✅ 지도가 'daegu_map_final.html'로 저장되었습니다.
🚀 브라우저를 실행합니다...
